In [3]:
!pip install nltk
!pip install sentence_transformers
!pip install kagglehub

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 61.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 294.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 670.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader


1. Download and read dataset.
   
It contains 3 tables - 'Users.csv', 'Ratings.csv', 'Books.csv'.

In [5]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("arashnic/book-recommendation-dataset")

print("Path to dataset files:", path)

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /home/jovyan/.cache/kagglehub/datasets/arashnic/book-recommendation-dataset/versions/3


In [6]:
 path = '/home/jovyan/.cache/kagglehub/datasets/arashnic/book-recommendation-dataset/versions/3'

In [7]:
import os

files = os.listdir(path)
print(files)

['DeepRec.png', 'Users.csv', 'Ratings.csv', 'Books.csv', 'classicRec.png', 'recsys_taxonomy2.png']


In [8]:

import pandas as pd

books = pd.read_csv(os.path.join(path, 'Books.csv'))
books


/tmp/ipykernel_118/1806486744.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv(os.path.join(path, 'Books.csv'))


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...


In [9]:
users = pd.read_csv(os.path.join(path, 'Users.csv'))
users.head(5)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [41]:
ratings = pd.read_csv(os.path.join(path, 'Ratings.csv'))
ratings

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


This script below processes book descriptions generated in different file from book dataset by cleaning the text, removing stopwords, and lemmatizing words to create a new preprocessed version of the descriptions. The script uses NLTK (Natural Language Toolkit) for text processing and applies the transformations to a Pandas DataFrame.

Steps taken for text processing:

-- convert text to lowercase: This ensures uniform processing and avoids treating "The" and "the" as different words.

-- remove special characters and numbers: Removes everything except letters and spaces (i.e., numbers, punctuation, special symbols).

-- tokenization: Splits text into individual words (tokens).

-- lemmatizer: Used to reduce words to their root form.

-- stop_words: Loads a set of common words that should be removed (e.g., "the", "is", "and", etc.).

In [11]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

nltk.download('punkt_tab')
import pandas as pd



processed_df = pd.read_pickle('books_w_descript.pkl')


# Download necessary NLTK resources
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("stopwords")

# Initialize lemmatizer and stop words
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

def preprocess_text(text):
    """Preprocess text: Lowercase, remove special chars, stop words, and lemmatize."""
    if not isinstance(text, str):  # Handle NaN cases
        return ""

    # Lowercase
    text = text.lower()

    # Remove special characters and numbers
    text = re.sub(r"[^a-zA-Z\s]", "", text)

    # Tokenization
    words = word_tokenize(text)

    # Remove stopwords and apply lemmatization
    cleaned_text = " ".join(lemmatizer.lemmatize(word) for word in words if word not in stop_words)

    return cleaned_text

processed_df['Description_vector'] = processed_df['Description'].apply(preprocess_text)

[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Code below applies TF-IDF (Term Frequency-Inverse Document Frequency) vectorization to preprocessed book descriptions, transforming text into a numerical representation suitable for machine learning models. TfidfVectorizer(...) creates a TF-IDF transformation object.

Parameters: max_features=5000: Limits the vocabulary to the top 5000 most frequent words (based on TF-IDF scores). Reduces dimensionality while retaining important features

Also, due to inconsistency, script ensures that only books present in both the ratings dataset and book descriptions are considered. It then maps book indices correctly, and finally converts the processed TF-IDF features into a PyTorch tensor for further use in machine learning models.

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Vectorize preprocessed descriptions
tfidf = TfidfVectorizer(stop_words="english", max_features=5000)


# Load book dataset and filter books that are in ratings
books_filtered = processed_df[processed_df["ISBN"].isin(ratings["ISBN"])].reset_index(drop=True)

# Create a mapping only for books that exist in both ratings and descriptions
book_map = {isbn: i for i, isbn in enumerate(books_filtered["ISBN"].unique())}

# Apply mapping to ratings (removing books that do not have descriptions)
ratings = ratings[ratings["ISBN"].isin(book_map)].copy()
ratings["Book-Index"] = ratings["ISBN"].map(book_map)

# Ensure that the number of books in ratings matches the number of books in book_desc_features
num_books = len(book_map)

# Generate new book_desc_features matrix that matches the book indices
tfidf_matrix = tfidf.fit_transform(books_filtered["Description_vector"])
print("TF-IDF Matrix Shape:", tfidf_matrix.shape)  # (num_books, num_features)

book_desc_features = torch.tensor(tfidf_matrix.toarray(), dtype=torch.float32)

# Assert to check sizes match
assert book_desc_features.shape[0] == num_books, "Mismatch in book description matrix size"


TF-IDF Matrix Shape: (270146, 5000)


In [13]:
# Create mappings (dictionary-based encoding)
user_map = {id: i for i, id in enumerate(ratings["User-ID"].unique())}

# Apply mappings
ratings["User-Index"] = ratings["User-ID"].map(user_map)

# Get total number of unique users and books
num_users = len(user_map)


# Train-test split
train, test = train_test_split(ratings, test_size=0.2, random_state=42)


In [28]:
train

,User-ID,ISBN,Book-Rating,Book-Index,User-Index
1018939,244298,3492232949,0,269449,81387
423934,101209,0451204026,0,228637,34236
975704,235105,0425147584,0,7163,78223
956372,231210,0440237025,0,9965,76865
931919,226031,0449126625,0,206937,75188
...,...,...,...,...,...
289631,69211,0312985207,9,6041,23796
409225,98391,0515132756,8,129056,33325
148815,33580,0446522856,0,43773,11808
750911,181944,0316602051,0,2415,60742


In [15]:
ratings_min = train["Book-Rating"].min()
ratings_max = train["Book-Rating"].max()

train["Book-Rating"] = (train["Book-Rating"] - ratings_min) / (ratings_max - ratings_min)
test["Book-Rating"] = (test["Book-Rating"] - ratings_min) / (ratings_max - ratings_min)



This code below defines a custom PyTorch Dataset (BookDataset) for training a machine learning model on book ratings, by using both user interactions and book descriptions.


In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import random_split


class BookDataset(Dataset):
    """PyTorch Dataset for Book Ratings including Book Descriptions"""
    def __init__(self, df, book_desc_features):
        self.users = torch.tensor(df["User-Index"].values, dtype=torch.long)
        self.books = torch.tensor(df["Book-Index"].values, dtype=torch.long)
        self.ratings = torch.tensor(df["Book-Rating"].values, dtype=torch.float32)

        # Ensure book indices are within bounds
        self.books = torch.clamp(self.books, 0, book_desc_features.shape[0] - 1)

        # Assign book descriptions safely
        self.book_desc_features = book_desc_features[self.books]

    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, idx):
        return self.users[idx], self.books[idx], self.book_desc_features[idx], self.ratings[idx]

# Update DataLoader
train_dataset = BookDataset(train, book_desc_features)
test_dataset = BookDataset(test, book_desc_features)

test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

# Split train dataset into training and validation sets
train_size = int(0.8 * len(train_dataset))  # 80% training
val_size = len(train_dataset) - train_size  # 20% validation
train_data, val_data = random_split(train_dataset, [train_size, val_size])

# Create new DataLoaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=256, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=256, shuffle=False)

Below is defined a Neural Collaborative Filtering (NCF) model that integrates book descriptions as additional features. The model is designed to predict user ratings for books using user embeddings, book embeddings, and book description embeddings.


The model converts IDs into dense 50D vectors using embeddings.

Converts TF-IDF features of Book Descriptions into dense vectors.

Merges User, Book, & Description Features.

Passes Through Fully Connected Layers predicting a single rating value.


In [17]:
import torch.nn.functional as F

class NCFWithDescription(nn.Module):
    def __init__(self, num_users, num_books, desc_dim, embedding_dim=50, dropout_rate=0.5):
        super(NCFWithDescription, self).__init__()

        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.book_embedding = nn.Embedding(num_books, embedding_dim)
        self.desc_fc = nn.Linear(desc_dim, embedding_dim)

        self.fc1 = nn.Linear(embedding_dim * 3, 64)
        self.fc2 = nn.Linear(64, 32)
        self.output = nn.Linear(32, 1)

        #self.relu = nn.ReLU() 
        self.leaky_relu = nn.LeakyReLU(0.2) # Use LeakyReLU with negative slope 0.2
        
        self.dropout = nn.Dropout(dropout_rate)  # Dropout layer

    def forward(self, user, book, book_desc):
        user_emb = self.user_embedding(user)
        book_emb = self.book_embedding(book)
        desc_emb = self.leaky_relu(self.desc_fc(book_desc))

        x = torch.cat([user_emb, book_emb, desc_emb], dim=1)

        x = self.leaky_relu(self.fc1(x))
        x = self.dropout(x)  # Apply dropout here
        x = self.leaky_relu(self.fc2(x))
        x = self.dropout(x)  # Apply dropout here
        x = self.output(x)

        return x.squeeze()

# Initialize model
embedding_dim = 50
desc_dim = book_desc_features.shape[1]  # Number of features in description embeddings
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = NCFWithDescription(num_users, num_books, desc_dim, embedding_dim, dropout_rate=0.3).to(device)


In [37]:
# Loss function & optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)  # Add weight decay for L2 regularization

# Learning rate scheduler: Reduce LR when validation loss stops improving
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)


best_val_loss = float("inf")
early_stopping_patience = 8
patience_counter = 0


# Training loop
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for users, books, book_descs, ratings in train_loader:
        users, books, book_descs, ratings = users.to(device), books.to(device), book_descs.to(device), ratings.to(device)

        # Forward pass
        predictions = model(users, books, book_descs)
        loss = criterion(predictions, ratings)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Compute average training loss
    avg_train_loss = total_loss / len(train_loader)

    # Validation Phase
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for users, books, book_descs, ratingss in val_loader:
            users, books, book_descs, ratingss = users.to(device), books.to(device), book_descs.to(device), ratingss.to(device)

            # Forward pass
            predictions = model(users, books, book_descs)
            loss = criterion(predictions, ratingss)

            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_loader)

    # Adjust learning rate based on validation loss
    scheduler.step(avg_val_loss)
    
    current_lr = optimizer.param_groups[0]['lr'] 

    # Check if validation loss improved
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0  # Reset patience counter
        torch.save(model.state_dict(), "best_model.pth")  # Save best model
    else:
        patience_counter += 1

    # Stop if validation loss increases 
    if patience_counter >= early_stopping_patience:
        print("Early stopping triggered.")
        break
        
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, LR: {current_lr:.6f}")


Epoch 1/30, Train Loss: 0.1550, Val Loss: 0.1449, LR: 0.000100
Epoch 2/30, Train Loss: 0.1462, Val Loss: 0.1427, LR: 0.000100
Epoch 3/30, Train Loss: 0.1432, Val Loss: 0.1406, LR: 0.000100
Epoch 4/30, Train Loss: 0.1404, Val Loss: 0.1379, LR: 0.000100
Epoch 5/30, Train Loss: 0.1375, Val Loss: 0.1350, LR: 0.000100
Epoch 6/30, Train Loss: 0.1340, Val Loss: 0.1316, LR: 0.000100
Epoch 7/30, Train Loss: 0.1300, Val Loss: 0.1278, LR: 0.000100
Epoch 8/30, Train Loss: 0.1255, Val Loss: 0.1236, LR: 0.000100
Epoch 9/30, Train Loss: 0.1208, Val Loss: 0.1197, LR: 0.000100
Epoch 10/30, Train Loss: 0.1167, Val Loss: 0.1168, LR: 0.000100
Epoch 11/30, Train Loss: 0.1134, Val Loss: 0.1150, LR: 0.000100
Epoch 12/30, Train Loss: 0.1108, Val Loss: 0.1140, LR: 0.000100
Epoch 13/30, Train Loss: 0.1087, Val Loss: 0.1136, LR: 0.000100
Epoch 14/30, Train Loss: 0.1065, Val Loss: 0.1136, LR: 0.000100
Epoch 15/30, Train Loss: 0.1032, Val Loss: 0.1144, LR: 0.000100
Epoch 16/30, Train Loss: 0.0983, Val Loss: 0.1164

In [40]:
import torch
import torch.nn as nn
import torch.optim as optim


checkpoint = torch.load('best_model.pth', map_location=device)


model = NCFWithDescription(num_users, num_books, desc_dim, embedding_dim, dropout_rate=0.3).to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)  # Add weight decay for L2 regularization

print(f"Model loaded")



model.eval()  # Set model to evaluation mode (important)
total_loss = 0

with torch.no_grad():  # Disable gradient calculations for evaluation
    for users, books, book_descs, ratingss in test_loader:
        users, books, book_descs, ratingss = users.to(device), books.to(device), book_descs.to(device), ratingss.to(device)

        # Forward pass (prediction)
        predictions = model(users, books, book_descs)
        loss = criterion(predictions, ratingss)  # Compute loss

        total_loss += loss.item()

    # Compute average test loss
    print(f"Test Loss: {total_loss/len(test_loader):.4f}")


/tmp/ipykernel_118/1495886655.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('best_model.pth', map_location=device)


Model loaded
Test Loss: 0.2267



Function defined below is returning top 5 recommended books for specified user.

Let's see what our model would recommend to read next for user '276729'

In [64]:


ratings["User-ID"] = ratings["User-ID"].astype(str)  # Convert User-ID to string
ratings1 = ratings[ratings["User-ID"] == "276729"]
ratings1

ratings1["ISBN"] = ratings1["ISBN"].astype(str)
books_filtered["ISBN"] = books_filtered["ISBN"].astype(str)

books_filtered1 = books_filtered[books_filtered["ISBN"].isin(ratings1["ISBN"])]

books_filtered1 = books_filtered1.to_dict(orient="records")  # Convert to list of dictionaries

books_filtered1
for book in books_filtered1:
    print(f"{book['Book-Title']} by {book['Book-Author']} (ISBN: {book['ISBN']}), Description: {book['Description']}")



Help!: Level 1 by Philip Prowse (ISBN: 052165615X), Description: This is the first book in the series that teaches young children the basics of English. It includes an introduction to the alphabet, the sounds of English, the letters of the alphabet, and the sounds of words. The book also includes a phonics section that teaches children to read words using the sounds of English. This book is ideal for children who are learning English as a second language.
The Amsterdam Connection : Level 4 (Cambridge English Readers) by Sue Leather (ISBN: 0521795028), Description: In this book, students will learn about the Amsterdam connection and how it affected the world. They will learn about the Amsterdam connection and how it affected the world. They will also learn about the history of Amsterdam. They will learn about the history of Amsterdam. They will also learn about the history of Amsterdam. They will also learn about the history of Amsterdam. They will also learn about the history of Amster

/tmp/ipykernel_118/2992480573.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings1["ISBN"] = ratings1["ISBN"].astype(str)


In [46]:
import numpy as np
import torch
import pandas as pd

def recommend_books_dl(user_id, n=5, filter_rated=True, batch_size=256):
    """Recommends books for a user using the trained NCF model with book descriptions and scaled ratings."""

    if user_id not in user_map:
        print(f"User {user_id} not found. Returning popular books instead.")
        return recommend_popular_books(n)

    model.eval()  

    user_idx = torch.tensor([user_map[user_id]] * num_books, dtype=torch.long).to(device)
    book_indices = torch.arange(num_books, dtype=torch.long)

    predicted_scores = []

    with torch.no_grad():
        for i in range(0, num_books, batch_size):  # Process books in batches
            batch_books = book_indices[i:i+batch_size].to(device)
            batch_book_descs = book_desc_features[i:i+batch_size].to(device)  # Move only a batch

            batch_scores = model(user_idx[i:i+batch_size], batch_books, batch_book_descs)
            predicted_scores.append(batch_scores.cpu())  # Move results back to CPU

    predicted_scores = torch.cat(predicted_scores).numpy().flatten()  # Convert to NumPy

    #print(predicted_scores.dtype)
    #print(predicted_scores[0:5])

    # use valid indices 
    top_books_idx = np.argsort(predicted_scores)[::-1][:n]  
    #top_books_idx = [idx for idx in top_books_idx if idx < num_books]  # Ensure within bounds

    # all ISBNs exist in `book_map`
    recommended_books_isbn = [list(book_map.keys())[i] for i in top_books_idx if i < len(book_map)]
    #print(recommended_books_isbn)

    # Avoid indexing errors when retrieving `Predicted-Rating`
    recommended_books_df = books_filtered [books_filtered ["ISBN"].isin(recommended_books_isbn)].copy()
    #recommended_books_df = books[books["ISBN"].isin(recommended_books_isbn)].copy()

    # Fix indexing to prevent errors
    recommended_books_df["Predicted-Rating"] = recommended_books_df["ISBN"].map(lambda isbn: predicted_scores[book_map[isbn]] if isbn in book_map else np.nan)

    if filter_rated:
        rated_books = ratings[ratings["User-ID"] == user_id]["ISBN"].tolist()
        recommended_books_df = recommended_books_df[~recommended_books_df["ISBN"].isin(rated_books)]

    return recommended_books_df[["Description", "ISBN", "Book-Title", "Book-Author", "Predicted-Rating"]].head(n).to_dict(orient="records")


recommendations_dl = recommend_books_dl(276729, 5)
for book in recommendations_dl:
    print(f"{book['Book-Title']} by {book['Book-Author']} (ISBN: {book['ISBN']}), Predicted Rating: {book['Predicted-Rating']:.2f}, Description: {book['Description']}")


Joshua by Joseph F Girzone (ISBN: 0020198906), Predicted Rating: 0.19, Description: Joshua is a story of the life of a man named Joshua who lived in the time of the Judges. He was the son of a man named Jacob, who was a shepherd in the land of Canaan. Joshua was a good shepherd, and he was very faithful to his sheep. One day, the Lord told Joshua to go to the land of Canaan and take it for the people of Israel. Joshua was very excited and he went to the land of Canaan, but he was not allowed to go into the land
Blind Man's Bluff: The Untold Story of American Submarine Espionage by Sherry Sontag (ISBN: 006103004X), Predicted Rating: 0.19, Description: 'In this fascinating account of a secret war, Sherry Sontag takes us on a gripping journey through the shadowy world of submarine espionage. In the 1960s and 70s, America's most elite Navy officers were engaged in a high-stakes game of cat and mouse with the Soviet Union's navy. The Soviets had a fleet of nuclear submarines, and the Americ